In [2]:
# Import Library Dependencies
from sqlalchemy import create_engine, inspect
from bs4 import BeautifulSoup as bs
from config import user, password

import requests as req
import pandas as pd
import json

In [3]:
# Web Scraping Function That Returns Desired Web Links For Each Team (<span>)
# [Function] Params: <String> Link Description as Shown on Website
          # Returns: <Dictionary> Team Name: Description Link
        
def espn_mlb_team_links(link_desc):
    _base_url = 'http://www.espn.com'
    _htmldoc = req.get(_base_url + '/mlb/teams').text
    _soup = bs(_htmldoc, 'html.parser')
    _span_tags = _soup.find_all('span', class_='TeamLinks__Link n9 nowrap')
    #_span_tags = _soup.find_all('span', class_='Table2__td--headshot')

    _team_urls = {}
    for _span in _span_tags:
        try:
            _link = _span.a['href'].strip()
            _info = _link.split('/')

            if f'{link_desc.strip()}' in _info:
                _team = _info[-1].strip().title()
                _team_url = _base_url + _link
                _team_urls[_team] = _team_url
                print(_team, '\n', _team_url)
        
        except TypeError as e:
            print('\n\t>> No Player Link Available <<\n')
            continue
            
    return _team_urls

In [4]:
# Web Scraping Function That Returns Basic Player Info & Links to Personal Page on ESPN (<span>)
# [Function] Params: <Dictionary> Team Roster
                    #Key: Team Name
                    #Value: Link to Team Roster
          # Returns: <Dictionary> Player Gen Info & Personal Page Links(ESPN)
        
def team_player_links(roster):
    _player_links = {}
    for _team, _url in roster.items():
        _htmldoc = req.get(_url).text
        _soup = bs(_htmldoc, 'html.parser')
        _span_tags = _soup.find_all('span', class_='Table2__td--headshot')
        
        for _span in _span_tags:
            try:
                _link = _span.a['href']
                _info = _link.split('/')
            
                if 'player' in _info:
                    _pID = _info[-1]
                    _pName = _span.a.find('figure').img['title'].strip().title()
                    _t = _url.split('/').index('name') + 1
                    _tID = _url.split('/')[_t].upper()
                    
                    if _pID not in _player_links:
                        _player_links[_pID] = {
                            'id': _pID,
                            'name': _pName,
                            'team': _team,
                            'tid': _tID,
                            'link': _link
                        }
                        print(f'ID: {_pID}, Name: {_pName}, Team: {_team}\n\tLink: {_link}')
 
            except TypeError as e1:
                print('\n<Skip> No Player Link Found\n')
            
            except AttributeError as e2:
                print('\n<Skip> No Player Link Found\n')
            
            except KeyError as e3:
                print('\n<Skip> No Player Link Found\n')
                
    return _player_links

In [5]:
# Web Scraping Function That Extracts Personal Player Information From ESPN (<li>)

def get_player_info(p_link):
    _htmldoc = req.get(p_link).text
    _soup = bs(_htmldoc, 'html.parser')
    _li_tags = _soup.find_all('li')
    try:
        _num = _li_tags[0].text.split(' ')[0][1:]
        _pos = _li_tags[0].text.split(' ')[1]
        _thr = _li_tags[1].text.split(',')[0][-1]
        _bat = _li_tags[1].text.split(',')[1][-1]
        _team = _li_tags[2].text
        _bday = _li_tags[3].text[10:].split('(')[0].strip()
        _age = _li_tags[3].text[10:].split('(')[1].split(':')[1][:-1].strip()
        _city = _li_tags[4].text[10:].split(',')[0].strip()
        _exp = _li_tags[5].text[10:]
        _col = _li_tags[6].text[7:]
        _hght = _li_tags[7].text.split(',')[0][5:].strip()
        _wght = _li_tags[7].text.split(',')[1].strip()
        
    except IndexError:
        return {
            'pid': None,
            'fname': None,
            'lname': None,
            'age': 0,
            'height': None,
            'weight': None,
            'birthday': None,
            'city': None,
            'number': -1,
            'position': None,
            'exp': None,
            'team': None,
            'tid': None,
            'throw': None,
            'bat': None,
            'college': None,
        }
    return {
        'pid': None,
        'fname': None,
        'lname': None,
        'age': int(_age),
        'height': _hght,
        'weight': _wght,
        'birthday': _bday,
        'city': _city,
        'number': int(_num),
        'position': _pos,
        'exp': _exp,
        'team': _team,
        'tid': None,
        'throw': _thr,
        'bat': _bat,
        'college': _col,
    }

In [6]:
def get_player_season_stats(p_name, p_link):
    _pName = p_name.replace(' ', '-').lower()
    _modLink = p_link.split('/')
    if 'player' in _modLink:
        _insIndex = _modLink.index('player') + 1
        _modLink.insert(_insIndex, 'stats')
        _modLink.append(_pName)
        _statsLink = '/'.join(modLink)
        
        pStats = pd.read_html(_statsLink)
        _sumStats = pStats[3]
        _colHead = _sumStats.iloc[1]
        _sumStats.columns = _colHead

In [8]:
# Return List of All MLB Team Roster Links
rosters = espn_mlb_team_links('roster')

Chicago-White-Sox 
 http://www.espn.com/mlb/team/roster/_/name/chw/chicago-white-sox
Cleveland-Indians 
 http://www.espn.com/mlb/team/roster/_/name/cle/cleveland-indians
Detroit-Tigers 
 http://www.espn.com/mlb/team/roster/_/name/det/detroit-tigers
Kansas-City-Royals 
 http://www.espn.com/mlb/team/roster/_/name/kc/kansas-city-royals
Minnesota-Twins 
 http://www.espn.com/mlb/team/roster/_/name/min/minnesota-twins
Baltimore-Orioles 
 http://www.espn.com/mlb/team/roster/_/name/bal/baltimore-orioles
Boston-Red-Sox 
 http://www.espn.com/mlb/team/roster/_/name/bos/boston-red-sox
New-York-Yankees 
 http://www.espn.com/mlb/team/roster/_/name/nyy/new-york-yankees
Tampa-Bay-Rays 
 http://www.espn.com/mlb/team/roster/_/name/tb/tampa-bay-rays
Toronto-Blue-Jays 
 http://www.espn.com/mlb/team/roster/_/name/tor/toronto-blue-jays
Houston-Astros 
 http://www.espn.com/mlb/team/roster/_/name/hou/houston-astros
Los-Angeles-Angels 
 http://www.espn.com/mlb/team/roster/_/name/laa/los-angeles-angels
Oakland-

In [9]:
# Return Dictionary Containing ALL MLB Player Gen Info & Link to Personal Page (ESPN)
players = team_player_links(rosters)


<Skip> No Player Link Found

ID: 31123, Name: Manny Banuelos, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/31123
ID: 36131, Name: Aaron Bummer, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/36131
ID: 34943, Name: Dylan Cease, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/34943
ID: 31255, Name: Alex Colome, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/31255
ID: 34850, Name: Jimmy Cordero, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/34850
ID: 33309, Name: Dylan Covey, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/33309
ID: 28895, Name: Ross Detwiler, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/28895
ID: 34240, Name: Jace Fry, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/34240
ID: 34913, Name: Carson Fulmer, Team: Chicago-White-Sox
	Link: http://www.espn.com/mlb/player/_/id/34913
ID: 32697, Name: Lucas Giolito, Team


<Skip> No Player Link Found

ID: 32156, Name: Scott Barlow, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/32156
ID: 34868, Name: Jacob Barnes, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/34868
ID: 31114, Name: Danny Duffy, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/31114
ID: 36077, Name: Heath Fillmyer, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/36077
ID: 33108, Name: Jesse Hahn, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/33108
ID: 37005, Name: Tim Hill, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/37005
ID: 36056, Name: Jakob Junis, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/36056
ID: 35292, Name: Brad Keller, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/35292
ID: 28864, Name: Ian Kennedy, Team: Kansas-City-Royals
	Link: http://www.espn.com/mlb/player/_/id/28864
ID: 33833, Name: Jorge Lopez, Tea


<Skip> No Player Link Found

ID: 32147, Name: Matt Barnes, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/32147
ID: 30189, Name: Ryan Brasier, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/30189
ID: 36631, Name: Colten Brewer, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/36631
ID: 30134, Name: Andrew Cashner, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/30134
ID: 30147, Name: Jhoulys Chacin, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/30147
ID: 31174, Name: Nathan Eovaldi, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/31174
ID: 40502, Name: Darwinzon Hernandez, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/40502
ID: 34400, Name: Brian Johnson, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/34400

<Skip> No Player Link Found

ID: 35297, Name: Travis Lakins, Team: Boston-Red-Sox
	Link: http://www.espn.com/mlb/player/_/id/35297
ID: 29966, Name:


<Skip> No Player Link Found

ID: 32145, Name: Jason Adam, Team: Toronto-Blue-Jays
	Link: http://www.espn.com/mlb/player/_/id/32145
ID: 32725, Name: Buddy Boshers, Team: Toronto-Blue-Jays
	Link: http://www.espn.com/mlb/player/_/id/32725
ID: 28855, Name: Clay Buchholz, Team: Toronto-Blue-Jays
	Link: http://www.espn.com/mlb/player/_/id/28855
ID: 31075, Name: Wilmer Font, Team: Toronto-Blue-Jays
	Link: http://www.espn.com/mlb/player/_/id/31075
ID: 33157, Name: Sam Gaviglio, Team: Toronto-Blue-Jays
	Link: http://www.espn.com/mlb/player/_/id/33157
ID: 32762, Name: Ken Giles, Team: Toronto-Blue-Jays
	Link: http://www.espn.com/mlb/player/_/id/32762
ID: 40947, Name: Anthony Kay, Team: Toronto-Blue-Jays
	Link: http://www.espn.com/mlb/player/_/id/40947
ID: 33139, Name: Derek Law, Team: Toronto-Blue-Jays
	Link: http://www.espn.com/mlb/player/_/id/33139
ID: 35773, Name: Tim Mayza, Team: Toronto-Blue-Jays
	Link: http://www.espn.com/mlb/player/_/id/35773
ID: 39709, Name: Thomas Pannone, Team: Toront


<Skip> No Player Link Found

ID: 30041, Name: Brett Anderson, Team: Oakland-Athletics
	Link: http://www.espn.com/mlb/player/_/id/30041
ID: 28668, Name: Homer Bailey, Team: Oakland-Athletics
	Link: http://www.espn.com/mlb/player/_/id/28668
ID: 33148, Name: Chris Bassitt, Team: Oakland-Athletics
	Link: http://www.espn.com/mlb/player/_/id/33148
ID: 32776, Name: Paul Blackburn, Team: Oakland-Athletics
	Link: http://www.espn.com/mlb/player/_/id/32776
ID: 32329, Name: Ryan Buchter, Team: Oakland-Athletics
	Link: http://www.espn.com/mlb/player/_/id/32329
ID: 32178, Name: Jake Diekman, Team: Oakland-Athletics
	Link: http://www.espn.com/mlb/player/_/id/32178
ID: 30773, Name: Mike Fiers, Team: Oakland-Athletics
	Link: http://www.espn.com/mlb/player/_/id/30773
ID: 31755, Name: Liam Hendriks, Team: Oakland-Athletics
	Link: http://www.espn.com/mlb/player/_/id/31755
ID: 33244, Name: Sean Manaea, Team: Oakland-Athletics
	Link: http://www.espn.com/mlb/player/_/id/33244
ID: 6470, Name: Yusmeiro Petit,


<Skip> No Player Link Found

ID: 39802, Name: Adbert Alzolay, Team: Chicago-Cubs
	Link: http://www.espn.com/mlb/player/_/id/39802
ID: 30564, Name: Tyler Chatwood, Team: Chicago-Cubs
	Link: http://www.espn.com/mlb/player/_/id/30564
ID: 30963, Name: Steve Cishek, Team: Chicago-Cubs
	Link: http://www.espn.com/mlb/player/_/id/30963
ID: 32055, Name: Yu Darvish, Team: Chicago-Cubs
	Link: http://www.espn.com/mlb/player/_/id/32055
ID: 6216, Name: Cole Hamels, Team: Chicago-Cubs
	Link: http://www.espn.com/mlb/player/_/id/6216
ID: 33173, Name: Kyle Hendricks, Team: Chicago-Cubs
	Link: http://www.espn.com/mlb/player/_/id/33173
ID: 30148, Name: Derek Holland, Team: Chicago-Cubs
	Link: http://www.espn.com/mlb/player/_/id/30148
ID: 32023, Name: Danny Hultzen, Team: Chicago-Cubs
	Link: http://www.espn.com/mlb/player/_/id/32023
ID: 30959, Name: Brandon Kintzler, Team: Chicago-Cubs
	Link: http://www.espn.com/mlb/player/_/id/30959
ID: 28487, Name: Jon Lester, Team: Chicago-Cubs
	Link: http://www.espn.c


<Skip> No Player Link Found

ID: 39813, Name: Dario Agrazal, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/39813
ID: 33870, Name: Steven Brault, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/33870
ID: 32171, Name: Kyle Crick, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/32171
ID: 33235, Name: Michael Feliz, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/33235
ID: 40865, Name: Geoff Hartlieb, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/40865
ID: 32827, Name: Clay Holmes, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/32827
ID: 33497, Name: Keone Kela, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/33497
ID: 33722, Name: Mitch Keller, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/33722
ID: 6211, Name: Francisco Liriano, Team: Pittsburgh-Pirates
	Link: http://www.espn.com/mlb/player/_/id/6211
ID: 34908, Name: Parke


<Skip> No Player Link Found

ID: 35241, Name: Sandy Alcantara, Team: Miami-Marlins
	Link: http://www.espn.com/mlb/player/_/id/35241
ID: 36110, Name: Jeff Brigham, Team: Miami-Marlins
	Link: http://www.espn.com/mlb/player/_/id/36110
ID: 32048, Name: Wei-Yin Chen, Team: Miami-Marlins
	Link: http://www.espn.com/mlb/player/_/id/32048
ID: 32669, Name: Adam Conley, Team: Miami-Marlins
	Link: http://www.espn.com/mlb/player/_/id/32669
ID: 39831, Name: Robert Dugger, Team: Miami-Marlins
	Link: http://www.espn.com/mlb/player/_/id/39831
ID: 33843, Name: Jarlin Garcia, Team: Miami-Marlins
	Link: http://www.espn.com/mlb/player/_/id/33843
ID: 33654, Name: Tayron Guerrero, Team: Miami-Marlins
	Link: http://www.espn.com/mlb/player/_/id/33654
ID: 39836, Name: Elieser Hernandez, Team: Miami-Marlins
	Link: http://www.espn.com/mlb/player/_/id/39836
ID: 40709, Name: Kyle Keller, Team: Miami-Marlins
	Link: http://www.espn.com/mlb/player/_/id/40709
ID: 34166, Name: Tyler Kinley, Team: Miami-Marlins
	Link: h


<Skip> No Player Link Found

ID: 33005, Name: Aaron Barrett, Team: Washington-Nationals
	Link: http://www.espn.com/mlb/player/_/id/33005
ID: 31313, Name: Patrick Corbin, Team: Washington-Nationals
	Link: http://www.espn.com/mlb/player/_/id/31313
ID: 30283, Name: Sean Doolittle, Team: Washington-Nationals
	Link: http://www.espn.com/mlb/player/_/id/30283
ID: 33152, Name: Roenis Elias, Team: Washington-Nationals
	Link: http://www.espn.com/mlb/player/_/id/33152
ID: 33793, Name: Erick Fedde, Team: Washington-Nationals
	Link: http://www.espn.com/mlb/player/_/id/33793
ID: 30505, Name: Javy Guerra, Team: Washington-Nationals
	Link: http://www.espn.com/mlb/player/_/id/30505
ID: 30506, Name: Jeremy Hellickson, Team: Washington-Nationals
	Link: http://www.espn.com/mlb/player/_/id/30506
ID: 30376, Name: Daniel Hudson, Team: Washington-Nationals
	Link: http://www.espn.com/mlb/player/_/id/30376
ID: 38582, Name: Tanner Rainey, Team: Washington-Nationals
	Link: http://www.espn.com/mlb/player/_/id/385


<Skip> No Player Link Found

ID: 30694, Name: Pedro Baez, Team: Los-Angeles-Dodgers
	Link: http://www.espn.com/mlb/player/_/id/30694
ID: 39251, Name: Walker Buehler, Team: Los-Angeles-Dodgers
	Link: http://www.espn.com/mlb/player/_/id/39251
ID: 41017, Name: Caleb Ferguson, Team: Los-Angeles-Dodgers
	Link: http://www.espn.com/mlb/player/_/id/41017
ID: 33852, Name: Dylan Floro, Team: Los-Angeles-Dodgers
	Link: http://www.espn.com/mlb/player/_/id/33852
ID: 32888, Name: Yimi Garcia, Team: Los-Angeles-Dodgers
	Link: http://www.espn.com/mlb/player/_/id/32888
ID: 38958, Name: Tony Gonsolin, Team: Los-Angeles-Dodgers
	Link: http://www.espn.com/mlb/player/_/id/38958
ID: 29630, Name: Kenley Jansen, Team: Los-Angeles-Dodgers
	Link: http://www.espn.com/mlb/player/_/id/29630
ID: 31992, Name: Joe Kelly, Team: Los-Angeles-Dodgers
	Link: http://www.espn.com/mlb/player/_/id/31992
ID: 28963, Name: Clayton Kershaw, Team: Los-Angeles-Dodgers
	Link: http://www.espn.com/mlb/player/_/id/28963
ID: 33130, Nam

In [10]:
# Iterate Through Each Player's Personal Page and Retrieve Detailed Player Info
mlb_player_data = []
for playerID, playerInfo in players.items():
    _pData = get_player_info(playerInfo['link'])
    _pData['pid'] = playerID
    _pData['fname'] = playerInfo['name'].split(' ')[0].strip()
    _pData['lname'] = playerInfo['name'].split(' ')[1].strip()
    _pData['tid'] = playerInfo['tid']
    mlb_player_data.append(_pData)
    print(json.dumps(_pData, indent=4, sort_keys=False), '\n')

{
    "pid": "31123",
    "fname": "Manny",
    "lname": "Banuelos",
    "age": 28,
    "height": "5-10",
    "weight": "215 lbs.",
    "birthday": "March 13, 1991",
    "city": "Durango",
    "number": 58,
    "position": "RP",
    "exp": "1 year",
    "team": "Chicago White Sox",
    "tid": "CHW",
    "throw": "L",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "36131",
    "fname": "Aaron",
    "lname": "Bummer",
    "age": 25,
    "height": "6-3",
    "weight": "200 lbs.",
    "birthday": "September 21, 1993",
    "city": "Valencia",
    "number": 39,
    "position": "RP",
    "exp": "2 years",
    "team": "Chicago White Sox",
    "tid": "CHW",
    "throw": "L",
    "bat": "L",
    "college": "Nebraska"
} 

{
    "pid": "34943",
    "fname": "Dylan",
    "lname": "Cease",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "CHW",
    "throw": null,

{
    "pid": "40393",
    "fname": "Danny",
    "lname": "Mendick",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "CHW",
    "throw": null,
    "bat": null,
    "college": null
} 

{
    "pid": "33984",
    "fname": "Yoan",
    "lname": "Moncada",
    "age": 24,
    "height": "6-2",
    "weight": "205 lbs.",
    "birthday": "May 27, 1995",
    "city": "Abreus",
    "number": 10,
    "position": "3B",
    "exp": "3 years",
    "team": "Chicago White Sox",
    "tid": "CHW",
    "throw": "B",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "32264",
    "fname": "Yolmer",
    "lname": "Sanchez",
    "age": 27,
    "height": "5-11",
    "weight": "185 lbs.",
    "birthday": "June 29, 1992",
    "city": "Maracay",
    "number": 5,
    "position": "2B",
    "exp": "5 years",
    "team": "Chicago White Sox",
    "tid": "CHW",
    "throw": "B",
    "bat": 

{
    "pid": "33267",
    "fname": "Nick",
    "lname": "Wittgren",
    "age": 28,
    "height": "6-2",
    "weight": "216 lbs.",
    "birthday": "May 29, 1991",
    "city": "Torrance",
    "number": 62,
    "position": "RP",
    "exp": "3 years",
    "team": "Cleveland Indians",
    "tid": "CLE",
    "throw": "R",
    "bat": "R",
    "college": "Purdue"
} 

{
    "pid": "36062",
    "fname": "Hunter",
    "lname": "Wood",
    "age": 26,
    "height": "6-1",
    "weight": "175 lbs.",
    "birthday": "August 12, 1993",
    "city": "Rogers",
    "number": 54,
    "position": "RP",
    "exp": "2 years",
    "team": "Cleveland Indians",
    "tid": "CLE",
    "throw": "R",
    "bat": "R",
    "college": "Howard"
} 

{
    "pid": "33452",
    "fname": "Eric",
    "lname": "Haase",
    "age": 26,
    "height": "5-10",
    "weight": "180 lbs.",
    "birthday": "December 18, 1992",
    "city": "Westland",
    "number": 38,
    "position": "C",
    "exp": "1 year",
    "team": "Cleveland Indians

{
    "pid": "41422",
    "fname": "David",
    "lname": "Mckay",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "DET",
    "throw": null,
    "bat": null,
    "college": null
} 

{
    "pid": "33199",
    "fname": "Daniel",
    "lname": "Norris",
    "age": 26,
    "height": "6-2",
    "weight": "185 lbs.",
    "birthday": "April 25, 1993",
    "city": "Johnson City",
    "number": 44,
    "position": "SP",
    "exp": "5 years",
    "team": "Detroit Tigers",
    "tid": "DET",
    "throw": "L",
    "bat": "L",
    "college": "None"
} 

{
    "pid": "33509",
    "fname": "Zac",
    "lname": "Reininger",
    "age": 26,
    "height": "6-3",
    "weight": "170 lbs.",
    "birthday": "January 28, 1993",
    "city": "San Antonio",
    "number": 26,
    "position": "RP",
    "exp": "2 years",
    "team": "Detroit Tigers",
    "tid": "DET",
    "throw": "R",
    

{
    "pid": "34868",
    "fname": "Jacob",
    "lname": "Barnes",
    "age": 29,
    "height": "6-2",
    "weight": "220 lbs.",
    "birthday": "April 14, 1990",
    "city": "St. Petersburg",
    "number": 50,
    "position": "RP",
    "exp": "3 years",
    "team": "Kansas City Royals",
    "tid": "KC",
    "throw": "R",
    "bat": "R",
    "college": "Florida Gulf Coast"
} 

{
    "pid": "31114",
    "fname": "Danny",
    "lname": "Duffy",
    "age": 30,
    "height": "6-3",
    "weight": "205 lbs.",
    "birthday": "December 21, 1988",
    "city": "Goleta",
    "number": 41,
    "position": "SP",
    "exp": "8 years",
    "team": "Kansas City Royals",
    "tid": "KC",
    "throw": "L",
    "bat": "L",
    "college": "None"
} 

{
    "pid": "36077",
    "fname": "Heath",
    "lname": "Fillmyer",
    "age": 25,
    "height": "6-1",
    "weight": "180 lbs.",
    "birthday": "May 16, 1994",
    "city": "Roebling",
    "number": 49,
    "position": "RP",
    "exp": "1 year",
    "team": 

...

{
    "pid": "30589",
    "fname": "Bryan",
    "lname": "Shaw",
    "age": 31,
    "height": "6-1",
    "weight": "232 lbs.",
    "birthday": "November 8, 1987",
    "city": "Livermore",
    "number": 29,
    "position": "RP",
    "exp": "8 years",
    "team": "Colorado Rockies",
    "tid": "COL",
    "throw": "R",
    "bat": "B",
    "college": "Long Beach State"
} 

{
    "pid": "39822",
    "fname": "Jesus",
    "lname": "Tinoco",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "COL",
    "throw": null,
    "bat": null,
    "college": null
} 

{
    "pid": "29436",
    "fname": "Drew",
    "lname": "Butera",
    "age": 36,
    "height": "6-1",
    "weight": "200 lbs.",
    "birthday": "August 9, 1983",
    "city": "Evansville",
    "number": 25,
    "position": "C",
    "exp": "9 years",
    "team": "Colorado Rockies",
    "tid": "COL",
    "throw": "R

{
    "pid": "29630",
    "fname": "Kenley",
    "lname": "Jansen",
    "age": 31,
    "height": "6-5",
    "weight": "265 lbs.",
    "birthday": "September 30, 1987",
    "city": "Willemstad",
    "number": 74,
    "position": "RP",
    "exp": "9 years",
    "team": "Los Angeles Dodgers",
    "tid": "LAD",
    "throw": "R",
    "bat": "B",
    "college": "None"
} 

{
    "pid": "31992",
    "fname": "Joe",
    "lname": "Kelly",
    "age": 31,
    "height": "6-1",
    "weight": "174 lbs.",
    "birthday": "June 9, 1988",
    "city": "Anaheim",
    "number": 17,
    "position": "RP",
    "exp": "7 years",
    "team": "Los Angeles Dodgers",
    "tid": "LAD",
    "throw": "R",
    "bat": "R",
    "college": "UC Riverside"
} 

{
    "pid": "28963",
    "fname": "Clayton",
    "lname": "Kershaw",
    "age": 31,
    "height": "6-4",
    "weight": "226 lbs.",
    "birthday": "March 19, 1988",
    "city": "Dallas",
    "number": 22,
    "position": "SP",
    "exp": "11 years",
    "team": "Los

{
    "pid": "31392",
    "fname": "Joc",
    "lname": "Pederson",
    "age": 27,
    "height": "6-1",
    "weight": "220 lbs.",
    "birthday": "April 21, 1992",
    "city": "Palo Alto",
    "number": 31,
    "position": "LF",
    "exp": "5 years",
    "team": "Los Angeles Dodgers",
    "tid": "LAD",
    "throw": "L",
    "bat": "L",
    "college": "None"
} 

{
    "pid": "30699",
    "fname": "A.J.",
    "lname": "Pollock",
    "age": 31,
    "height": "6-1",
    "weight": "195 lbs.",
    "birthday": "December 5, 1987",
    "city": "Hebron",
    "number": 11,
    "position": "CF",
    "exp": "7 years",
    "team": "Los Angeles Dodgers",
    "tid": "LAD",
    "throw": "R",
    "bat": "R",
    "college": "Notre Dame"
} 

{
    "pid": "32900",
    "fname": "Chris",
    "lname": "Taylor",
    "age": 29,
    "height": "6-1",
    "weight": "196 lbs.",
    "birthday": "August 29, 1990",
    "city": "Virginia Beach",
    "number": 3,
    "position": "LF",
    "exp": "5 years",
    "team": "L

{
    "pid": "31097",
    "fname": "Manny",
    "lname": "Machado",
    "age": 27,
    "height": "6-3",
    "weight": "215 lbs.",
    "birthday": "July 6, 1992",
    "city": "Miami",
    "number": 13,
    "position": "3B",
    "exp": "7 years",
    "team": "San Diego Padres",
    "tid": "SD",
    "throw": "R",
    "bat": "R",
    "college": "None"
} 

{
    "pid": "32800",
    "fname": "Seth",
    "lname": "Mejias-Brean",
    "age": 0,
    "height": null,
    "weight": null,
    "birthday": null,
    "city": null,
    "number": -1,
    "position": null,
    "exp": null,
    "team": null,
    "tid": "SD",
    "throw": null,
    "bat": null,
    "college": null
} 

{
    "pid": "35383",
    "fname": "Luis",
    "lname": "Urias",
    "age": 22,
    "height": "5-9",
    "weight": "160 lbs.",
    "birthday": "June 3, 1997",
    "city": "Sonora",
    "number": 9,
    "position": "2B",
    "exp": "1 year",
    "team": "San Diego Padres",
    "tid": "SD",
    "throw": "R",
    "bat": "R",
    

{
    "pid": "33714",
    "fname": "Aramis",
    "lname": "Garcia",
    "age": 26,
    "height": "6-2",
    "weight": "195 lbs.",
    "birthday": "January 12, 1993",
    "city": "Hialeah",
    "number": 16,
    "position": "C",
    "exp": "1 year",
    "team": "San Francisco Giants",
    "tid": "SF",
    "throw": "R",
    "bat": "R",
    "college": "FIU"
} 

{
    "pid": "30112",
    "fname": "Buster",
    "lname": "Posey",
    "age": 32,
    "height": "6-1",
    "weight": "210 lbs.",
    "birthday": "March 27, 1987",
    "city": "Leesburg",
    "number": 28,
    "position": "C",
    "exp": "10 years",
    "team": "San Francisco Giants",
    "tid": "SF",
    "throw": "R",
    "bat": "R",
    "college": "Florida State"
} 

{
    "pid": "31032",
    "fname": "Stephen",
    "lname": "Vogt",
    "age": 34,
    "height": "6-0",
    "weight": "225 lbs.",
    "birthday": "November 1, 1984",
    "city": "Visalia",
    "number": 21,
    "position": "C",
    "exp": "6 years",
    "team": "San Fr

In [11]:
# Convert <Dictionary> 'mlb_player_data' to DataFrame
col_order = [
    'pid',
    'fname',
    'lname',
    'age',
    'height',
    'weight',
    'birthday',
    'city',
    'number',
    'position',
    'exp',
    'team',
    'throw',
    'bat',
    'college'
]
mlb_players_df = pd.DataFrame(mlb_player_data)[col_order]
mlb_players_df.head()

,pid,fname,lname,age,height,weight,birthday,city,number,position,exp,team,throw,bat,college
0,31123,Manny,Banuelos,28,5-10,215 lbs.,"March 13, 1991",Durango,58,RP,1 year,Chicago White Sox,L,R,None
1,36131,Aaron,Bummer,25,6-3,200 lbs.,"September 21, 1993",Valencia,39,RP,2 years,Chicago White Sox,L,L,Nebraska
2,34943,Dylan,Cease,0,None,None,None,None,-1,None,None,None,None,None,None
3,31255,Alex,Colome,30,6-1,220 lbs.,"December 31, 1988",Santo Domingo,48,RP,6 years,Chicago White Sox,R,R,None
4,34850,Jimmy,Cordero,27,6-4,222 lbs.,"October 19, 1991",San Cristobal,60,RP,1 year,Chicago White Sox,R,R,None


In [16]:
mlb_players_df.head(15)

,pid,fname,lname,age,height,weight,birthday,city,number,position,exp,team,throw,bat,college
0,31123,Manny,Banuelos,28,5-10,215 lbs.,"March 13, 1991",Durango,58,RP,1 year,Chicago White Sox,L,R,None
1,36131,Aaron,Bummer,25,6-3,200 lbs.,"September 21, 1993",Valencia,39,RP,2 years,Chicago White Sox,L,L,Nebraska
2,34943,Dylan,Cease,0,None,None,None,None,-1,None,None,None,None,None,None
3,31255,Alex,Colome,30,6-1,220 lbs.,"December 31, 1988",Santo Domingo,48,RP,6 years,Chicago White Sox,R,R,None
4,34850,Jimmy,Cordero,27,6-4,222 lbs.,"October 19, 1991",San Cristobal,60,RP,1 year,Chicago White Sox,R,R,None
5,33309,Dylan,Covey,28,6-2,195 lbs.,"August 14, 1991",Glendale,68,SP,2 years,Chicago White Sox,R,R,San Diego
6,28895,Ross,Detwiler,33,6-5,210 lbs.,"March 6, 1986",St. Louis,54,SP,10 years,Chicago White Sox,L,R,Missouri State
7,34240,Jace,Fry,26,6-1,190 lbs.,"July 9, 1993",Beaverton,57,RP,2 years,Chicago White Sox,L,L,Oregon State
8,34913,Carson,Fulmer,25,6-0,195 lbs.,"December 13, 1993",Lakeland,51,SP,3 years,Chicago White Sox,R,R,Vanderbilt
9,32697,Lucas,Giolito,25,6-6,245 lbs.,"July 14, 1994",Santa Monica,27,SP,3 years,Chicago White Sox,R,R,None


In [12]:
# Establish Connection to MySQL Database
MySQL_DB_Connection = f'{user}:{password}@localhost/moneyball_db'
engine = create_engine(f'mysql://{MySQL_DB_Connection}', echo=True)
engine.table_names()

2019-09-08 19:27:26,633 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2019-09-08 19:27:26,635 INFO sqlalchemy.engine.base.Engine ()
2019-09-08 19:27:26,642 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2019-09-08 19:27:26,643 INFO sqlalchemy.engine.base.Engine ()
2019-09-08 19:27:26,650 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2019-09-08 19:27:26,652 INFO sqlalchemy.engine.base.Engine ()
2019-09-08 19:27:26,655 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2019-09-08 19:27:26,656 INFO sqlalchemy.engine.base.Engine ()
2019-09-08 19:27:26,661 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2019-09-08 19:27:26,662 INFO sqlalchemy.engine.base.Engine ()
2019-09-08 19:27:26,665 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2019-09-08 19:27:26,666 INFO sqlalchemy.engine.base.E

['mlb_players_2019', 'mlb_teams_2019', 'player_stats_2019']

In [13]:
# Import MLB Player Data to MySQL Database
mlb_players_df.to_sql(
    name='mlb_players_2019', 
    con=engine, if_exists='append', 
    index=False
)

2019-09-08 19:27:26,756 INFO sqlalchemy.engine.base.Engine DESCRIBE `mlb_players_2019`
2019-09-08 19:27:26,757 INFO sqlalchemy.engine.base.Engine ()
2019-09-08 19:27:26,766 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-08 19:27:26,793 INFO sqlalchemy.engine.base.Engine INSERT INTO mlb_players_2019 (pid, fname, lname, age, height, weight, birthday, city, number, position, exp, team, throw, bat, college) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2019-09-08 19:27:26,795 INFO sqlalchemy.engine.base.Engine (('31123', 'Manny', 'Banuelos', 28, '5-10', '215 lbs.', 'March 13, 1991', 'Durango', 58, 'RP', '1 year', 'Chicago White Sox', 'L', 'R', 'None'), ('36131', 'Aaron', 'Bummer', 25, '6-3', '200 lbs.', 'September 21, 1993', 'Valencia', 39, 'RP', '2 years', 'Chicago White Sox', 'L', 'L', 'Nebraska'), ('34943', 'Dylan', 'Cease', 0, None, None, None, None, -1, None, None, None, None, None, None), ('31255', 'Alex', 'Colome', 30, '6-1', '220 lbs.', 'December 

In [14]:
# Export MLB Player DataFrame to CSV w/ Ordered Columns
mlb_players_df.to_csv(r'2019-MLB_Player_Data.csv', index=False)

In [15]:
print("\n>>> [Moneyball] Data Aggregation & Database Import -- Complete <<<\n")


>>> [Moneyball] Data Aggregation & Database Import -- Complete <<<

